##### Copyright 2023 Google LLC

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IllvdSBhcmUgYW4gZXhwZXJ0IHJlZ2V4IHN0cmluZyBjcmVhdG9yIGFuZCB1bmRlcnN0YW5kIGhvdyByZWdleCB3b3Jrcy4gWW91ciBqb2IgaXMgdG8gY29udmVydCB0aGUgdXNlcidzIG5hdHVyYWwgbGFuZ3VhZ2UgcXVlcmllcyBhbmQgY29uc3RyYWludHMgaW4gdGhlIGZvcm0gb2YgcmVnZXguIEFmdGVyIGdlbmVyYXRpbmcgdGhlIHJlZ2V4IHN0cmluZywgcHJvdmlkZSBleHBsYW5hdGlvbiBpbiBkZXRhaWwgd2l0aCBhIGZldyBleGFtcGxlcy4gVGhlbiBkZW1vbnN0cmF0ZSBpdHMgdXNlIGluIGEgcHl0aG9uIGNvZGUuXFxuXFxuVXNlciBxdWVyeTpcXG5HaXZlIG1lIHRoZSByZWdleCBlcXVpdmFsZW50IG9mIHRoZSBmb2xsb3dpbmc6XFxuTXkgcmVxdWlyZW1lbnRzIGFyZTpcXG4xLiBJIHdhbnQgbXkgc3RyaW5nIHRvIGhhdmUgMi02MyBjaGFyYWN0ZXJzLlxcbjIuIFRoZSBzdHJpbmcgc2hvdWxkIGJlIGFscGhhbnVtZXJpYyBhbmQgY2FuIGNvbnRhaW4gLSBhbHNvLlxcbjMuIFRoZSBzdHJpbmcgbXVzdCBzdGFydCBhcyB3ZWxsIGFzIGVuZCB3aXRoIGFscGhhbnVtZXJpYyBjaGFyYWN0ZXJzIG9ubHkuIn0seyJ0ZXh0IjoiIyEkXFxuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiIyMgUmVnZXggZm9yIFVzZXIncyBSZXF1aXJlbWVudHNcXG5cXG5IZXJlJ3MgdGhlIHJlZ2V4IHN0cmluZyBiYXNlZCBvbiB0aGUgdXNlcidzIHJlcXVpcmVtZW50czpcXG5cXG5gYGByZWdleFxcbl5bYS16QS1aMC05XXsyLDYzfVthLXpBLVowLTldJFxcbmBgYFxcblxcbiMjIyBFeHBsYW5hdGlvbjpcXG5cXG4xLiAqKmBeYCoqOiBUaGlzIGFuY2hvcnMgdGhlIHJlZ2V4IHRvIHRoZSBiZWdpbm5pbmcgb2YgdGhlIHN0cmluZy4gVGhpcyBlbnN1cmVzIHRoZSBwYXR0ZXJuIG1hdGNoZXMgb25seSBpZiBpdCBzdGFydHMgYXQgdGhlIGJlZ2lubmluZy5cXG5cXG4yLiAqKmBbYS16QS1aMC05XWAqKjogVGhpcyBjaGFyYWN0ZXIgY2xhc3MgbWF0Y2hlcyBhbnkgYWxwaGFudW1lcmljIGNoYXJhY3RlciAtIGJvdGggdXBwZXJjYXNlIGFuZCBsb3dlcmNhc2UgbGV0dGVycywgYXMgd2VsbCBhcyBudW1iZXJzLlxcblxcbjMuICoqYHsyLDYzfWAqKjogVGhpcyBxdWFudGlmaWVyIHNwZWNpZmllcyB0aGF0IHRoZSBwcmVjZWRpbmcgY2hhcmFjdGVyIGNsYXNzIChgW2EtekEtWjAtOV1gKSBtdXN0IG9jY3VyIGJldHdlZW4gMiBhbmQgNjMgdGltZXMgKGluY2x1c2l2ZSkuXFxuXFxuNC4gKipgW2EtekEtWjAtOV1gKio6IFRoaXMgY2hhcmFjdGVyIGNsYXNzIGlzIGFnYWluIHVzZWQgdG8gbWF0Y2ggYW4gYWxwaGFudW1lcmljIGNoYXJhY3Rlci4gVGhpcyBwYXJ0IGVuc3VyZXMgdGhlIHN0cmluZyBlbmRzIHdpdGggYW4gYWxwaGFudW1lcmljIGNoYXJhY3Rlci5cXG5cXG41LiAqKmAkYCoqOiBUaGlzIGFuY2hvcnMgdGhlIHJlZ2V4IHRvIHRoZSBlbmQgb2YgdGhlIHN0cmluZy4gSXQgZW5zdXJlcyB0aGUgcGF0dGVybiBtYXRjaGVzIG9ubHkgaWYgaXQgZW5kcyBhdCB0aGUgZW5kIG9mIHRoZSBpbnB1dCBzdHJpbmcuXFxuXFxuIyMjIEV4YW1wbGVzOlxcblxcbnwgSW5wdXQgU3RyaW5nIHwgTWF0Y2hlcyB8IEV4cGxhbmF0aW9uIHxcXG58LS0tfC0tLXwtLS18XFxufCAqKmFiYzEyMyoqIHwg4pyFIHwgTWF0Y2hlcyAtIDYgYWxwaGFudW1lcmljIGNoYXJhY3RlcnMsIHN0YXJ0cyBhbmQgZW5kcyB3aXRoIGFscGhhbnVtZXJpYyBjaGFyYWN0ZXJzLiB8XFxufCAqKkhlbGxvX3dvcmxkKiogfCDinYwgfCBEb2Vzbid0IG1hdGNoIC0gQ29udGFpbnMgYW4gdW5kZXJzY29yZSBjaGFyYWN0ZXIgKGBfYCkgd2hpY2ggaXMgbm90IGFsbG93ZWQuIHxcXG58ICoqYSoqIHwg4p2MIHwgRG9lc24ndCBtYXRjaCAtIFN0cmluZyBpcyBsZXNzIHRoYW4gMiBjaGFyYWN0ZXJzIGxvbmcuIHxcXG58ICoqMTIzNDU2Nzg5MDEyMzQ1Njc4OTAxMjM0NTY3ODkwMTIzNDU2Nzg5MCoqIHwg4p2MIHwgRG9lc24ndCBtYXRjaCAtIFN0cmluZyBpcyBsb25nZXIgdGhhbiA2MyBjaGFyYWN0ZXJzLiB8XFxufCAqKjEyMy1hYmMqKiB8IOKdjCB8IERvZXNuJ3QgbWF0Y2ggLSBBbHRob3VnaCB3aXRoaW4gbGVuZ3RoIHJhbmdlIGFuZCBjb250YWluaW5nIGFsbG93ZWQgY2hhcmFjdGVycywgaXQgZW5kcyB3aXRoIGAtYCB3aGljaCBpcyBub3QgYWxsb3dlZC4gfFxcblxcblxcbiMjIyBQeXRob24gQ29kZTpcXG5cXG5gYGBweXRob25cXG5pbXBvcnQgcmVcXG5cXG5yZWdleF9wYXR0ZXJuID0gclxcXCJeW2EtekEtWjAtOV17Miw2M31bYS16QS1aMC05XSRcXFwiXFxuXFxudGVzdF9zdHJpbmdzID0gW1xcXCJhYmMxMjNcXFwiLCBcXFwiSGVsbG9fd29ybGRcXFwiLCBcXFwiYVxcXCIsIFxcXCIxMjM0NTY3ODkwMTIzNDU2Nzg5MDEyMzQ1Njc4OTAxMjM0NTY3ODkwXFxcIiwgXFxcIjEyMy1hYmNcXFwiLCBcXFwiYWJjMTIzLXh5elxcXCIsIFxcXCJhYkMxMjNcXFwiXVxcblxcbmZvciBzdHJpbmcgaW4gdGVzdF9zdHJpbmdzOlxcbiAgbWF0Y2ggPSByZS5tYXRjaChyZWdleF9wYXR0ZXJuLCBzdHJpbmcpXFxuICBpZiBtYXRjaDpcXG4gICAgcHJpbnQoZlxcXCIne3N0cmluZ30nIC0gTWF0Y2hlcyFcXFwiKVxcbiAgZWxzZTpcXG4gICAgcHJpbnQoZlxcXCIne3N0cmluZ30nIC0gRG9lc24ndCBNYXRjaFxcXCIpXFxuYGBgXFxuXFxuVGhpcyBjb2RlIGl0ZXJhdGVzIHRocm91Z2ggYSBsaXN0IG9mIHN0cmluZ3MgYW5kIHVzZXMgdGhlIGByZS5tYXRjaGAgZnVuY3Rpb24gdG8gY2hlY2sgaWYgZWFjaCBzdHJpbmcgbWF0Y2hlcyB0aGUgZGVmaW5lZCByZWdleCBwYXR0ZXJuLiBUaGUgcmVzdWx0IGlzIHByaW50ZWQgZm9yIGVhY2ggc3RyaW5nLCBpbmRpY2F0aW5nIHdoZXRoZXIgaXQgbWF0Y2hlZCBvciBub3QuIFxcbiJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MiwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "You are an expert regex string creator and understand how regex works. Your job is to convert the user's natural language queries and constraints in the form of regex. After generating the regex string, provide explanation in detail with a few examples. Then demonstrate its use in a python code.\\n\\nUser query:\\nGive me the regex equivalent of the following:\\nMy requirements are:\\n1. I want my string to have 2-63 characters.\\n2. The string should be alphanumeric and can contain - also.\\n3. The string must start as well as end with alphanumeric characters only."
            },
            {
                "text": "#!$\\n"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "## Regex for User's Requirements\\n\\nHere's the regex string based on the user's requirements:\\n\\n```regex\\n^[a-zA-Z0-9]{2,63}[a-zA-Z0-9]$\\n```\\n\\n

## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [3]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

You are an expert regex string creator and understand how regex works. Your job is to convert the user's natural language queries and constraints in the form of regex. After generating the regex string, provide explanation in detail with a few examples. Then demonstrate its use in a python code.\n\nUser query:\nGive me the regex equivalent of the following:\nMy requirements are:\n1. I want my string to have 2-63 characters.\n2. The string should be alphanumeric and can contain - also.\n3. The string must start as well as end with alphanumeric characters only. 

#!$\n 

-------------------------------------------------------------------------------- 

Role: model 

## Regex for User's Requirements\n\nHere's the regex string based on the user's requirements:\n\n```regex\n^[a-zA-Z0-9]{2,63}[a-zA-Z0-9]$\n```\n\n### Explanation:\n\n1. **`^`**: This anchors the regex to the beginning of the string. This ensures the pattern matches only if it starts at the beginning.\n\n2. **`[a-